In [ ]:
! pip install votekit

# Math of VoteKit

This notebook is a collection of live demos of VoteKit. This is intended to focus on the underlying math of VoteKit, and should be used as a companion to the "Math of VoteKit" presentation.